## What is Reinforcement Learning

- the goal is to find a function that maps the **state (s)** to an **action (a)**
- uses a reward function to train model **R(s)**

Applications
- Controlling robots
- factory optimization
- financial (stock) trading
- playing games (including video games)

### The Return in Reinforcement Learning
-  **discount factor (γ)**: modifies reward credited to each step, discounting rewards further in the future (often a number close to 1)
- the **return** in reinforcement learning is the **sum of the rewards the system gets** but **weighted by the discount factor** -> rewards in the future are weighted by the discount factor raised to a higher power

### Making Decisions: Policies in Reinforcement Learning
- a policy function, **π(s) = a**, tells you what **action (a)** to take in a given **state (s)**

- **goal**: fund a policy that tells you what action to take in every state (s) so as to maximize the return


### Markov Decision Process (MDP)
- model for sequential decision making when outcomes are uncertain and partly controllable
- "Markov" means that the future only depends on the current state

### State-Action Value Function (Q Function, Q*. Optimal Q Function)

- a function typicall denoted by **Q(s, a)**
    - gives a number equal to the return if you start in a **state (s)**, take the **action (a)** once, and behave optimally after that
    - tells us how good it is to take action a in state s
    - the best possible return from **state (s)** is **max Q(s, a)**